In [1]:
!pip install --upgrade pip

ERROR: To modify pip, please run the following command:
C:\Users\adidn\miniconda3\envs\dp4ai\python.exe -m pip install --upgrade pip



  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


In [2]:
!pip install wapiti3

In [3]:
!wapiti --version


     __      __               .__  __  .__________
    /  \    /  \_____  ______ |__|/  |_|__\_____  \
    \   \/\/   /\__  \ \____ \|  \   __\  | _(__  <
     \        /  / __ \|  |_> >  ||  | |  |/       \
      \__/\  /  (____  /   __/|__||__| |__/______  /
           \/        \/|__|                      \/
Wapiti 3.2.8 (wapiti-scanner.github.io)
3.2.8


In [ ]:
# !pip install ipython

# Helper Function for Viewing Scan Reports Within the Notebook

The following helper function fetches the html file of the report and displays it within python.

In [34]:
import os
from IPython.display import display, HTML
import glob
import pathlib

def find_html_report(out):
    # If out is a file and ends with .html, use it
    if os.path.isfile(out) and out.lower().endswith('.html'):
        return os.path.abspath(out)
    # If out is a directory, search for the newest .html inside it
    if os.path.isdir(out):
        matches = glob.glob(os.path.join(out, "*.html"))
        if matches:
            # choose the most recent file
            matches.sort(key=os.path.getmtime, reverse=True)
            return os.path.abspath(matches[0])
    # If out doesn't exist as provided, search for any html file pattern that looks like wapiti output
    candidates = glob.glob(out + "*/*.html") + glob.glob(out + "*.html")
    if candidates:
        candidates.sort(key=os.path.getmtime, reverse=True)
        return os.path.abspath(candidates[0])
    return None

report_path = find_html_report(out_name)

def display_report(out):
    if find_html_report(out):
        print("Report file found:", find_html_report(out))
        try:
            with open(find_html_report(out), 'r', encoding='utf-8') as f:
                html = f.read()
            display(HTML(html))
        except Exception as e:
            print("Could not render HTML inline (error):", e)
            print("Open the file in your browser instead:", find_html_report(out))
    else:
        print("No HTML report found. Check that the scan produced the report and that 'out_name' matches the -o argument passed to Wapiti.")

# Scanning Gruyere

We use the following command to scan the gruyere instance for vulnerabilities.

In [32]:
!wapiti -u https://google-gruyere.appspot.com/574248707168921554278463496334824873101/ -o gruyere_scan --flush-session


 ██╗    ██╗ █████╗ ██████╗ ██╗████████╗██╗██████╗
 ██║    ██║██╔══██╗██╔══██╗██║╚══██╔══╝██║╚════██╗
 ██║ █╗ ██║███████║██████╔╝██║   ██║   ██║ █████╔╝
 ██║███╗██║██╔══██║██╔═══╝ ██║   ██║   ██║ ╚═══██╗
 ╚███╔███╔╝██║  ██║██║     ██║   ██║   ██║██████╔╝
  ╚══╝╚══╝ ╚═╝  ╚═╝╚═╝     ╚═╝   ╚═╝   ╚═╝╚═════╝  
Wapiti 3.2.8 (wapiti-scanner.github.io)
CSP is not set for URL: https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
X-Frame-Options is not set on https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
X-Content-Type-Options is not set on https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
Strict-Transport-Security is not set on https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
HttpOnly flag is not set on the cookie 'GRUYERE' set at 'https://google-gruyere.appspot.com/574248707168921554278463496334824873101/login?uid=default&pw=Letm3in_'
Secure flag is not set on the cookie: 'GRUYERE' set at 

In [35]:
display_report('gruyere_scan')

Report file found: c:\Users\adidn\Documents\Uni\Year 3\Networks and Systerm Security\Workshop 5\gruyere_scan\google-gruyere.appspot.com_12102025_1022.html


Category,Number of vulnerabilities found
Backup file,0
Cleartext Submission of Password,0
Weak credentials,0
CRLF Injection,0
Content Security Policy Configuration,1
Cross Site Request Forgery,0
Potentially dangerous file,0
Command execution,0
Path Traversal,0
Fingerprint web application framework,0


# Focused Scan

We can focus the scan to see only the vunerabilities for specific high impact attacks, such as XSS or SQL injection.

In [36]:
!wapiti -u https://google-gruyere.appspot.com/574248707168921554278463496334824873101/ -m xss,sql -o gruyere_xss_sql --flush-session


     __      __               .__  __  .__________
    /  \    /  \_____  ______ |__|/  |_|__\_____  \
    \   \/\/   /\__  \ \____ \|  \   __\  | _(__  <
     \        /  / __ \|  |_> >  ||  | |  |/       \
      \__/\  /  (____  /   __/|__||__| |__/______  /
           \/        \/|__|                      \/
Wapiti 3.2.8 (wapiti-scanner.github.io)
[*] Saving scan state, please wait...

[*] Launching module sql

[*] Launching module xss
---
Reflected Cross Site Scripting in https://google-gruyere.appspot.com/574248707168921554278463496334824873101/snippets.gtl via injection in the parameter uid
Evil request:
    GET /574248707168921554278463496334824873101/snippets.gtl?uid=%3CScRiPt%3Ealert%28%27wpvi7krfqi%27%29%3C%2FsCrIpT%3E HTTP/1.1
    host: google-gruyere.appspot.com
    connection: keep-alive
    user-agent: Mozilla/5.0 (Windows NT 6.1; rv:45.0) Gecko/20100101 Firefox/45.0
    accept-language: en-US
    accept-encoding: gzip, deflate, br
    accept: text/html,application/xhtml

In [40]:
display_report('gruyere_xss_sql')

Report file found: c:\Users\adidn\Documents\Uni\Year 3\Networks and Systerm Security\Workshop 5\gruyere_xss_sql\google-gruyere.appspot.com_12102025_1059.html


Category,Number of vulnerabilities found
Backup file,0
Cleartext Submission of Password,0
Weak credentials,0
CRLF Injection,0
Content Security Policy Configuration,0
Cross Site Request Forgery,0
Potentially dangerous file,0
Command execution,0
Path Traversal,0
Fingerprint web application framework,0


# Deep Scan

You can tune the depth to tell wapiti to click links up to 2 levels deep. This increased depth allows us to find more hidden vulnerabilities.

In [37]:
!wapiti -u https://google-gruyere.appspot.com/574248707168921554278463496334824873101/ --depth 2 --scope folder -o gruyere_deep --flush-session


     __      __               .__  __  .__________
    /  \    /  \_____  ______ |__|/  |_|__\_____  \
    \   \/\/   /\__  \ \____ \|  \   __\  | _(__  <
     \        /  / __ \|  |_> >  ||  | |  |/       \
      \__/\  /  (____  /   __/|__||__| |__/______  /
           \/        \/|__|                      \/
Wapiti 3.2.8 (wapiti-scanner.github.io)
CSP is not set for URL: https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
X-Frame-Options is not set on https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
X-Content-Type-Options is not set on https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
Strict-Transport-Security is not set on https://google-gruyere.appspot.com/574248707168921554278463496334824873101/
Potential full path disclosure in https://google-gruyere.appspot.com/574248707168921554278463496334824873101/snippets.gtl?uid=..%2F..%2F..%2F..%2F..%2F..%2F: /../../../../../../../../../../../../../../../../../.

In [41]:
display_report('gruyere_deep')

Report file found: c:\Users\adidn\Documents\Uni\Year 3\Networks and Systerm Security\Workshop 5\gruyere_deep\google-gruyere.appspot.com_12102025_1105.html


Category,Number of vulnerabilities found
Backup file,0
Cleartext Submission of Password,0
Weak credentials,0
CRLF Injection,0
Content Security Policy Configuration,1
Cross Site Request Forgery,0
Potentially dangerous file,0
Command execution,0
Path Traversal,0
Fingerprint web application framework,0


# Authenticated Scan

Google Gruyere has features that are only available when logged in. To scan those, we give Wapiti a session cookie. I got the session cookie from chrome dev tools. I am also telling Wapiti to scan only for XSS, SQL injection, command execution and file inclusion, to make the scan faster.

In [ ]:
!wapiti -u https://google-gruyere.appspot.com/574248707168921554278463496334824873101/ -H "Cookie: GRUYERE=59455721|hmmmm||author" -m xss,sql,exec,file -o gruyere_auth_scan --flush-session


 ██╗    ██╗ █████╗ ██████╗ ██╗████████╗██╗██████╗
 ██║    ██║██╔══██╗██╔══██╗██║╚══██╔══╝██║╚════██╗
 ██║ █╗ ██║███████║██████╔╝██║   ██║   ██║ █████╔╝
 ██║███╗██║██╔══██║██╔═══╝ ██║   ██║   ██║ ╚═══██╗
 ╚███╔███╔╝██║  ██║██║     ██║   ██║   ██║██████╔╝
  ╚══╝╚══╝ ╚═╝  ╚═╝╚═╝     ╚═╝   ╚═╝   ╚═╝╚═════╝  
Wapiti 3.2.8 (wapiti-scanner.github.io)


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\adidn\miniconda3\envs\dp4ai\Scripts\wapiti.exe\__main__.py", line 7, in <module>
  File "C:\Users\adidn\miniconda3\envs\dp4ai\Lib\site-packages\wapitiCore\main\wapiti.py", line 513, in wapiti_asyncio_wrapper
    asyncio.run(wapiti_main())
  File "C:\Users\adidn\miniconda3\envs\dp4ai\Lib\asyncio\runners.py", line 194, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\adidn\miniconda3\envs\dp4ai\Lib\asyncio\runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\adidn\miniconda3\envs\dp4ai\Lib\asyncio\base_events.py", line 687, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "C:\Users\adidn\miniconda3\envs\dp4ai\Lib\site-packages\wapitiCore\main\wapiti.py", line 404, in wapiti_main
    

In [ ]:
display_report('gruyere_auth')